In [1]:
%%time
import pandas as pd
import tensorflow as tf
import numpy as np
#import matplotlib.pyplot as plt
#from numpy import loadtxt
from keras.utils import np_utils
#from sklearn.preprocessing import LabelEncoder


from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import word_tokenize

Wall time: 3.88 s


In [2]:
%%time
df = pd.read_excel("BQ Analysis v1_1.xlsx", "Project BQ")

Wall time: 10.3 s


In [3]:
%%time
df = df.astype(str)
df["PREPROCESS_TEXT"] = df["BQ_TRADE"] + "," + df["BQ_HEADING"] + "," + df["BQ_SUB_HEADING"] + "," + df["BQ_ITEM_HEADING"] + "," + df["BQ_ITEM_DESCRIPTION"] #+ "," + df["AC_TRADE_CODE"]
#df['RESULT'] = ""
#training_df = df.loc[df['CONTRACT_CODE'] == "FL49"].copy() // not enough training data?
df = df[df['AC_TRADE_CODE'] != "DA"]
df = df[df['AC_TRADE_CODE'] != "IP"]
df = df[df['AC_TRADE_CODE'] != "SS"]
df = df[df['AC_TRADE_CODE'] != "EL"]
df = df[df['AC_TRADE_CODE'] != "SO"]
df = df[df['AC_TRADE_CODE'] != "AS"]


df["PREPROCESS_TEXT"] = list(map(lambda x: x.lower(), list(df["PREPROCESS_TEXT"])))





Wall time: 349 ms


In [4]:
class_names = ['PS',
'ST',
'SM',
'EL',
'PL',
'BS',
'DR',
'CJ',
'PA',
'AR',
'PR',
'SO',
'GL',
'IF',
'DW',
'EX',
'IR',
'FD',
'AP',
'SU',
'FF',
'SG',
'AS',
'EF',
'BR',
'SS',
'DA',
'IP'
]





In [5]:
training_df = df[df['CONTRACT_CODE'] != "QHP3"].copy()


training_df["AC_TRADE_CODE_INDEX"] = list(map(lambda x: class_names.index(x), list(training_df.AC_TRADE_CODE)))
#training_df["AC_TRADE_CODE"].value_counts()
#training_df["AC_TRADE_CODE_INDEX"].value_counts()

In [6]:
test_df = df[df['CONTRACT_CODE'] == "QHP3"].copy()
test_df["AC_TRADE_CODE_INDEX"] = list(map(lambda x: class_names.index(x), list(test_df.AC_TRADE_CODE)))
test_df["AC_TRADE_CODE_INDEX"].value_counts()

0     2788
1     2712
2     2258
4      853
6      479
8      427
7      368
9      215
12      95
10      58
24      35
16      24
15      10
Name: AC_TRADE_CODE_INDEX, dtype: int64

In [7]:
# print(len(df["AC_TRADE_CODE"]))
# label_encoder = LabelEncoder()
# label_encoder.fit(df["AC_TRADE_CODE"])
# training_label = np_utils.to_categorical((label_encoder.transform(df["AC_TRADE_CODE"])))

# print(len(training_label))

In [8]:
# %%time
# tokenized_doc = []
# for d in training_df["PREPROCESS_TEXT"]:
#     tokenized_doc.append(word_tokenize(d))
# tokenized_doc
# tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_doc)]
# len(tagged_data)

In [9]:
%%time
model= Doc2Vec.load("test_doc2vec.model")
#training_text = model.wv.vectors
#print(len(training_text))

Wall time: 264 ms


In [10]:
%%time
# test_data = word_tokenize(df["PREPROCESS_TEXT"])
# v1 = model.infer_vector(test_data)
# print("V1_infer", v1)

training_text = []
for index, row in training_df.iterrows():
    training_text.append(model.infer_vector(word_tokenize(row["PREPROCESS_TEXT"])))

training_text = np.asarray(training_text)  
print(len(training_text))
print(training_text[0])





test_text = []
for index, row in test_df.iterrows():
    #print(model.infer_vector(word_tokenize(row["PREPROCESS_TEXT"])))
    test_text.append(model.infer_vector(word_tokenize(row["PREPROCESS_TEXT"])))

test_text = np.asarray(test_text)  
print(len(test_text))
print(test_text[0])


31075
[-0.06272464 -0.10067155 -0.4152371  -0.6165283  -0.91595304  0.29419473
 -0.52807826 -0.9100595  -1.4919195  -0.6168335  -0.48975003  0.31695592
 -0.11166995 -1.4265729   0.64957315  0.89637214 -0.15140072  0.7948094
  0.82698345  1.0828694 ]
10322
[ 0.4904189   1.2882067  -1.1763126   0.5540634   0.4928528  -1.4700594
 -0.10427852  0.00327017 -0.38436523 -0.1642787   0.9855542   0.6602815
  1.7499673   0.1199621   0.3647239  -0.643064   -0.7047786   1.0265734
 -0.31217182  2.3166337 ]
Wall time: 6min 5s


In [11]:
training_label = np.asarray(training_df["AC_TRADE_CODE_INDEX"])  
test_label = np.asarray(test_df["AC_TRADE_CODE_INDEX"])  

In [12]:
training_text.shape, training_label.shape


((31075, 20), (31075,))

In [13]:

%%time
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(20, )),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(50)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(training_text, training_label, epochs=100)


Epoch 1/100
972/972 [==============================] - 1s 771us/step - loss: 1.4826 - accuracy: 0.6383
Epoch 2/100
972/972 [==============================] - 1s 736us/step - loss: 0.5793 - accuracy: 0.8259
Epoch 3/100
972/972 [==============================] - 1s 725us/step - loss: 0.4927 - accuracy: 0.8481
Epoch 4/100
972/972 [==============================] - 1s 730us/step - loss: 0.4568 - accuracy: 0.8624
Epoch 5/100
972/972 [==============================] - 1s 734us/step - loss: 0.4142 - accuracy: 0.8752
Epoch 6/100
972/972 [==============================] - 1s 718us/step - loss: 0.3805 - accuracy: 0.8854
Epoch 7/100
972/972 [==============================] - 1s 728us/step - loss: 0.3483 - accuracy: 0.8939
Epoch 8/100
972/972 [==============================] - 1s 723us/step - loss: 0.3377 - accuracy: 0.8969
Epoch 9/100
972/972 [==============================] - 1s 725us/step - loss: 0.3229 - accuracy: 0.9026
Epoch 10/100
972/972 [==============================] - 1s 725us/step - l

972/972 [==============================] - 1s 717us/step - loss: 0.1046 - accuracy: 0.9658
Epoch 80/100
972/972 [==============================] - 1s 718us/step - loss: 0.1053 - accuracy: 0.9657
Epoch 81/100
972/972 [==============================] - 1s 713us/step - loss: 0.1030 - accuracy: 0.9688
Epoch 82/100
972/972 [==============================] - 1s 713us/step - loss: 0.1028 - accuracy: 0.9676
Epoch 83/100
972/972 [==============================] - 1s 711us/step - loss: 0.1030 - accuracy: 0.9687
Epoch 84/100
972/972 [==============================] - 1s 766us/step - loss: 0.1054 - accuracy: 0.9667
Epoch 85/100
972/972 [==============================] - 1s 817us/step - loss: 0.0966 - accuracy: 0.9698
Epoch 86/100
972/972 [==============================] - 1s 739us/step - loss: 0.1062 - accuracy: 0.9658
Epoch 87/100
972/972 [==============================] - 1s 713us/step - loss: 0.1007 - accuracy: 0.9697
Epoch 88/100
972/972 [==============================] - 1s 714us/step - loss:

In [14]:
test_loss, test_acc = model.evaluate(test_text,  test_label, verbose=2)

print('\nTest accuracy:', test_acc)

323/323 - 0s - loss: 0.4141 - accuracy: 0.9078

Test accuracy: 0.9077697992324829


In [15]:
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(test_text)

In [16]:

i = 0
test_df["PREDICT_RESULT"] = 0
for index, row in test_df.iterrows():
    test_df["PREDICT_RESULT"][index] = np.argmax(predictions[i])
    i = i + 1


test_df["COMPARE"] = np.where(test_df["AC_TRADE_CODE_INDEX"] == test_df["PREDICT_RESULT"], True, False)


<ipython-input-16-7c42d552e199>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["PREDICT_RESULT"][index] = np.argmax(predictions[i])


In [17]:
test_df['COMPARE'].value_counts()

True     9370
False     952
Name: COMPARE, dtype: int64